출처: https://wikidocs.net/262582

## api_key & lang_smith

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
from langchain_teddynote import logging

logging.langsmith("agent-tools")

LangSmith 추적을 시작합니다.
[프로젝트명]
agent-tools


In [3]:
import warnings

warnings.filterwarnings("ignore") # 경고 메시지 무시

## PythonREPLTool

In [4]:
from langchain_experimental.tools import PythonREPLTool

python_tool = PythonREPLTool() # 파이썬 코드를 실행하는 도구를 생성합니다.

In [5]:
# 파이썬 코드를 실행하고 결과를 반환합니다.
print(python_tool.invoke("print(100 + 200)"))

Python REPL can execute arbitrary code. Use with caution.


300



In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda


def print_and_execute(code, debug=True):
    if debug:
        print("CODE:")
        print(code)
    return python_tool.invoke(code)


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are Raymond Hetting, an expert python programmer, well versed in meta-programming and elegant, concise and short but well documented code. You follow the PEP8 style guide. "
            "Return only the code, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the code.",
        ),
        ("human", "{input}"),
    ]
)
# LLM 모델 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 프롬프트와 LLM 모델을 사용하여 체인 생성
chain = prompt | llm | StrOutputParser() | RunnableLambda(print_and_execute)


In [ ]:
# 결과 출력
print(chain.invoke("로또 번호 생성기를 출력하는 코드를 작성하세요.")) # if __name__ == "__main__": 때문에 print() 실행 x

CODE:
import random

def generate_lotto_numbers():
    """Generate a set of 6 unique lotto numbers from 1 to 45."""
    return sorted(random.sample(range(1, 46), 6))

if __name__ == "__main__":
    print(generate_lotto_numbers())



In [12]:
import random

def generate_lotto_numbers():
    """Generate a set of 6 unique lotto numbers from 1 to 45."""
    return sorted(random.sample(range(1, 46), 6))

if __name__ == "__main__":
    print(generate_lotto_numbers())

[3, 6, 23, 32, 35, 41]


## 검색 API 도구

- max_results (int): 반환할 최대 검색 결과 수 (기본값: 5)
- search_depth (str): 검색 깊이 ("basic" 또는 "advanced")
- include_domains (List[str]): 검색 결과에 포함할 도메인 목록
- exclude_domains (List[str]): 검색 결과에서 제외할 도메인 목록
- include_answer (bool): 원본 쿼리에 대한 짧은 답변 포함 여부
- include_raw_content (bool): 각 사이트의 정제된 HTML 콘텐츠 포함 여부
- include_images (bool): 쿼리 관련 이미지 목록 포함 여부

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

# 도구 생성
tool = TavilySearchResults(
    max_results=6,
    include_answer=True,
    include_raw_content=True,
    # include_images=True,
    # search_depth="advanced", # or "basic"
    include_domains=["github.io", "wikidocs.net"],
    # exclude_domains = []
)


In [4]:
# 도구 실행
tool.invoke({"query": "LangChain Tools 에 대해서 알려주세요"})

[{'url': 'https://wikidocs.net/262582',
  'content': "LangChain 에서 기본 제공하는 도구를 사용하여 쉽게 도구를 활용할 수 있으며, 사용자 정의 도구(Custom Tool) 를 쉽게 구축하는 것도 가능합니다. LangChain 한국어 튜토리얼\\n바로가기 👀\\n[LangChain] 에이전트(Agent)와 도구(tools)를 활용한 지능형 검색 시스템 구축 가이드\\n2024년 02월 09일\\n41 분 소요\\n이 글에서는 LangChain 의 Agent 프레임워크를 활용하여 복잡한 검색과 📍 전체 템플릿 코드\\n다음의 추적 링크에서 자세한 단계별 수행 결과를 확인할 수 있습니다\\nLangSmith 추적\\n마무리입니다!\\n 문서 기반 QA 시스템 설계 방법 - 심화편\\n2024년 02월 06일\\n22 분 소요\\nLangChain의 RAG 시스템을 통해 문서(PDF, txt, 웹페이지 등)에 대한 질문-답변을 찾는 과정을 정리하였습니다.\\n'},  {'url': 'https://wikidocs.net/234282',  'content': 'Mar 19, 2024 · langchain 은 언어 모델과 관련된 다양한 기능을 제공하는 라이브러리로, 이 중 검색 도구 생성 기능은 데이터 검색 및 처리 작업을 용이하게 한다.'}] 이 도구를 사용하면 DALL-E API를 쉽게 통합하여 텍스트 기반 이미지 생성 기능을 구현할 수 있습니다."},
 {'url': 'https://teddylee777.github.io/langchain/langchain-agent/',
  'content': '태그:\nAgent,\nAPI KEY,\nFAISS,\nLangChain,\nLangSmith,\nmemory,\nOpenAI,\nPython,\nRetriever,\nTavily Search,\ntools,\n검색도구,\n랭체인,\n에이전트\n카테고리:\nlangchain\n업데이트: 2024년 02월 09일\n참고\n

### 기본 검색 설정

- query (str): 검색하고자 하는 키워드나 문장
- search_depth (str): 검색의 상세도. "basic"(기본) 또는 "advanced"(고급) 중 선택
- topic (str): 검색 주제 분야. "general"(일반) 또는 "news"(뉴스) 중 선택
- days (int): 검색 결과의 최신성. 지정된 일수 이내의 결과만 반환
- max_results (int): 반환받을 최대 검색 결과 수

### 도메인 필터링

- include_domains (list): 검색 결과에 반드시 포함할 도메인 목록
- exclude_domains (list): 검색 결과에서 제외할 도메인 목록

### 결과 상세 설정

- include_answer (bool): API가 생성한 답변 포함 여부
- include_raw_content (bool): 웹페이지의 원본 HTML 콘텐츠 포함 여부
- include_images (bool): 관련 이미지 정보 포함 여부
- format_output (bool): 검색 결과의 포맷팅 적용 여부

### 기타

- **kwargs: 추가적인 키워드 인자. API의 향후 업데이트나 특수 기능에 사용될 수 있음

In [6]:
from langchain_teddynote.tools.tavily import TavilySearch

# 기본 예제
tavily_tool = TavilySearch()

# include_domains 사용 예제
# 특정 도메인만 포함하여 검색
tavily_tool_with_domains = TavilySearch(include_domains=["github.io", "naver.com"])

# exclude_domains 사용 예제
# 특정 도메인을 제외하고 검색
tavily_tool_exclude = TavilySearch(exclude_domains=["ads.com", "spam.com"])

# 다양한 파라미터를 사용한 검색 예제
result1 = tavily_tool.search(
    query="유튜버 테디노트에 대해서 알려줘",  # 검색 쿼리
    search_depth="advanced",  # 고급 검색 수준
    topic="general",  # 일반 주제
    days=7,  # 최근 7일 내 결과
    max_results=10,  # 최대 10개 결과
    include_answer=True,  # 답변 포함
    include_raw_content=True,  # 원본 콘텐츠 포함
    include_images=True,  # 이미지 포함
    format_output=True,  # 결과 포맷팅
)

# 뉴스 검색 예제
result2 = tavily_tool.search(
    query="최신 AI 기술 동향",  # 검색 쿼리
    search_depth="basic",  # 기본 검색 수준
    topic="news",  # 뉴스 주제
    days=3,  # 최근 3일 내 결과
    max_results=5,  # 최대 5개 결과
    include_answer=False,  # 답변 미포함
    include_raw_content=False,  # 원본 콘텐츠 미포함
    include_images=False,  # 이미지 미포함
    format_output=True,  # 결과 포맷팅
)

# 특정 도메인 포함 검색 예제
result3 = tavily_tool_with_domains.search(
    query="파이썬 프로그래밍 팁",  # 검색 쿼리
    search_depth="advanced",  # 고급 검색 수준
    max_results=3,  # 최대 3개 결과
)

# 특정 도메인 제외 검색 예제
result4 = tavily_tool_exclude.search(
    query="건강한 식단",  # 검색 쿼리
    search_depth="basic",  # 기본 검색 수준
    days=30,  # 최근 30일 내 결과
    max_results=7,  # 최대 7개 결과
)

# 결과 출력
print("기본 검색 결과:", result1)
print("뉴스 검색 결과:", result2)
print("특정 도메인 포함 검색 결과:", result3)
print("특정 도메인 제외 검색 결과:", result4)


기본 검색 결과: ['<document><title>유튜브 채널 2차 수익화 (약 3개월 소요) - 네이버 블로그</title><url>https://blog.naver.com/PostView.naver?blogId=teddysecretnote&logNo=223443681263</url><content>블로그 테디의 유튜브 비밀노트 블로그 검색 블로그 블로그 공지 ‘테디의 비밀노트’ 유튜브 채널 소개 (유튜브 왕초보를 위한 유튜브 채널 육성 꿀팁 전달) 유튜브 비밀노트 유튜브 채널 2차 수익화 (약 3개월 소요) 왕초보를 위한 유튜브 육성 채널 지난 번 유튜브 채널 1차 수익화 글에서 유튜브 채널 1차 수익화 (약 2개월 소요) 안녕하세요 왕초보를 위한 유튜브 육성 채널 테디의 비밀노트를 운영중인 테디 입니다 :) \'왕초보 유... ■ YouTube 채널 (테디의 비밀노트) 유튜브 채널 5개 실패 후 6번째 도전에서 성공해 유튜브 조회수로만 약 8000만원의 수입을 올린 8만 구독자 유튜버가 되었습니다. \\"나도 유튜브 한 번 해보고 싶은데\\" 라고 마음을 먹었지만 도무지 어떻게 해야할 지 감이 안 잡히시는 분들, 왕초보분들을 대상으로 제 경험담을 하나하나 공유해 드리겠습니다. 테디의 유튜브 비밀노트 이 블로그 전체 카테고리 글</content><raw>NAVER\n블로그\n테디의 유튜브 비밀노트\n블로그 검색\n이 블로그에서 검색\n공감 \n첫 댓글을 남겨보세요 \n공유하기\n메뉴 바로가기\n본문 바로가기\n내 블로그 \n이웃블로그 \n블로그 홈 \n로그인\n사용자 링크\n블로그 메뉴\n 블로그\n안부\n블로그\n공지 목록\n공지글\n글 제목    작성일\n공지 ‘테디의 비밀노트’ 유튜브 채널 소개 (유튜브 왕초보를 위한 유튜브 채널 육성 꿀팁 전달)\n    2024. 5. 6.\n전체보기 20개의 글\n전체보기\n목록열기\n유튜브 비밀노트\n유튜브 채널 2차 수익화 (약 3개월 소요)\n유튜버 테디 ・ 2024. 5. 11. 21:28\nURL 복사

## Image 생성 도구 (DALL-E)

### 주요 속성

- model: 사용할 DALL-E 모델 이름 (기본값: "dall-e-2", "dall-e-3")
- n: 생성할 이미지 수 (기본값: 1)
- size: 생성할 이미지 크기
- "dall-e-2": "1024x1024", "512x512", "256x256"
- "dall-e-3": "1024x1024", "1792x1024", "1024x1792"
- style: 생성될 이미지의 스타일 (기본값: "natural", "vivid")
- quality: 생성될 이미지의 품질 (기본값: "standard", "hd")
- max_retries: 생성 시 최대 재시도 횟수

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.9, max_tokens=1000)

# DALL-E 이미지 생성을 위한 프롬프트 템플릿 정의
prompt = PromptTemplate.from_template(
    "Generate a detailed IMAGE GENERATION prompt for DALL-E based on the following description. "
    "Return only the prompt, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the prompt"
    "Output should be less than 1000 characters. Write in English only."
    "Image Description: \n{image_desc}",
)

# 프롬프트, LLM, 출력 파서를 연결하는 체인 생성
chain = prompt | llm | StrOutputParser()

# 체인 실행
image_prompt = chain.invoke(
    {"image_desc": "스마트폰을 바라보는 사람들을 풍자한 neo-classicism painting"}
)

# 이미지 프롬프트 출력
print(image_prompt)


Create a neo-classical painting that humorously depicts a group of people intently gazing at their smartphones. In a grand, classical setting reminiscent of ancient Greek or Roman architecture, show figures dressed in toga-like garments, embodying a blend of traditional and modern aesthetics. Their expressions should range from fascination to bewilderment as they interact with their devices. Include classical elements such as columns and statues in the background, juxtaposed with contemporary smartphones to emphasize the contrast between the two eras. The color palette should be soft and warm, with a focus on light and shadow to enhance the dramatic effect, capturing the irony of timelessness in the age of technology.


In [8]:
# DALL-E API 래퍼 가져오기
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from IPython.display import Image

# DALL-E API 래퍼 초기화
# model: 사용할 DALL-E 모델 버전
# size: 생성할 이미지 크기
# quality: 이미지 품질
# n: 생성할 이미지 수
dalle = DallEAPIWrapper(model="dall-e-3", size="1024x1024", quality="standard", n=1)

# 질문
query = "스마트폰을 바라보는 사람들을 풍자한 neo-classicism painting"

# 이미지 생성 및 URL 받기
# chain.invoke()를 사용하여 이미지 설명을 DALL-E 프롬프트로 변환
# dalle.run()을 사용하여 실제 이미지 생성
# image_url = dalle.run(chain.invoke({"image_desc": query}))
image_url = dalle.run(image_prompt)

# 생성된 이미지를 표시합니다.
Image(url=image_url, width=500)

## 사용자 정의 도구(Custom Tool)

### @tool 데코레이터

In [9]:
from langchain.tools import tool


# 데코레이터를 사용하여 함수를 도구로 변환합니다.
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b


@tool
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

In [10]:
# 도구 실행
add_numbers.invoke({"a": 3, "b": 4})

7

In [11]:
# 도구 실행
multiply_numbers.invoke({"a": 3, "b": 4})

12

### 구글 뉴스기사 검색 도구

API 키가 필요하지 않습니다. (RSS 피드를 사용하기 때문)

In [12]:
from langchain_teddynote.tools import GoogleNews

# 도구 생성
news_tool = GoogleNews()

In [13]:
# 최신 뉴스 검색
news_tool.search_latest(k=5)

[{'url': 'https://news.google.com/rss/articles/CBMikAFBVV95cUxPX1hMUnR0aGJTOGtvaE9fS01DYVZkeW5ZbzJuNG8zMDdxWnp6U0dVMi1OUkxzeHZLbDJ3MUIzbzNma3Z6RTRINUE1Q3JWQl93UFcwYXc3bUtRUjZBVFlxc09XTDhxLTVhRllpQmVJLVcwVzEyNjB6bldxY0Uwa0ZTRERjYVdGbTdrTWE4SVdVREY?oc=5',
  'content': '윤 대통령 "반국가세력 반드시 척결" 비상 계엄 선포 [전문] - 조선일보'},
 {'url': 'https://news.google.com/rss/articles/CBMiZEFVX3lxTFBuS29uMm1OSmUwblBMUmxMWFlKbmNoaVZBNTIwWEh3RHB0eWNNLVFGSU13N09hOWtQRVgtTUN2VXYyd3pfT01peVNrVVdiSUFWblhneDdHaVJ5WE9SZVllODRETW0?oc=5',
  'content': '홍준표 “명태균·강혜경 여론조작 고소…조사 의뢰한 적 없어” - 한겨레'},
 {'url': 'https://news.google.com/rss/articles/CBMihgFBVV95cUxPd0I0Tll0eFdvRmxPU1ZoZzhibU45enlHME52QWFQYWpNU2NGcXVYT0FBNUk2ampSNEg4RFZCOFo5VF94bkplZzd0bkRqa2JmLVAtNGZ0a3FGMUVwOEN0UC00TFZKNDl3dzh4NENULS1tTEtoY2RhU3IzbEt5QXV3TUpwSWMtQdIBmgFBVV95cUxONnBWMTlnNG8zNVhyZXA5azF3Ul9ySEZzWUt1ZFg3eTlkS2tJeUtRaUZLVmE5dE1oSkREMmZmMW54Ynp4aTZ5QllDdGM2SFk4ai1QRmQySC1FWmMzajZyWVBMb3hpSHlNVncyUEduTjBtOHl1MDdpNmFQX0FTZ2ljSi1MQ2hRSzNCSG91X1pUaGM2SFVyc

In [14]:
# 키워드로 뉴스 검색
news_tool.search_by_keyword("AI 투자", k=5)

[{'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTE44RmFKdm8yMVBQVmFIc0xFMldLTWg1X0x5dGhsWmNYcU53T0hSUmt4N0RYRnJGZHlkZUk2MGdHdkVDamJBSkg3OEpYVk54VnlCWG9QRlZCckJfZ9IBVEFVX3lxTE1yNmgwR19HQXlFckVFOGJqSk9wMEpHRDJnbGZYUWFZOUNmUHEzRGl3bXZqazFWM3hybmk3cFY1SUV1alAyMzhPUl9FNEY1NXpTT2psMw?oc=5',
  'content': "부자들은 다 안다…'1억으로 7700만원 버는' 대박 투자 - 한국경제"},
 {'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTFBWaTJrbXEweXdGYlRHa0N5bTZUNEdEM09JRVZKOFdUX1RRaW40QmtsSlJMVk44MXV4dV9jSno1bTU4VWxqSzV1UlFJeW5qWExweVAwWVFCZUpNZ9IBVkFVX3lxTE1zSXVFOVRDSm1oRFJJeEZ2Rm5tMXVTX1ZuNDlka1RpNDA0MlpCX0dCLUhUYUFpRG84VDZyVGJjaFlrcS15SVhRVWRhcnB0OWxqdlVZS1pB?oc=5',
  'content': '과기정통부, 2026년 AI 투자 방향 제시…차세대 AI 기술 개발 투자 박차 - 아주경제'},
 {'url': 'https://news.google.com/rss/articles/CBMiTkFVX3lxTE9peUV6VW14bF9oWWNFSUdjNHk5eHMzcENreWVqZEVQaG1HWmJFUEhscHNtc19yZ0lfb0JPMEVneUdaVmVtT0FxNXEyMlUyZw?oc=5',
  'content': "AI 민간투자 3위 이스라엘, '버티컬 AI'에 집중 투자 - 전자신문"},
 {'url': 'https://news.google.com/rss/articles/CBMikAFBVV

In [15]:
from langchain_teddynote.tools import GoogleNews
from langchain.tools import tool
from typing import List, Dict


# 키워드로 뉴스 검색하는 도구 생성
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""
    print(query)
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)

In [16]:
# 실행 결과
search_keyword.invoke({"query": "AI 투자"})

AI 투자


[{'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTE44RmFKdm8yMVBQVmFIc0xFMldLTWg1X0x5dGhsWmNYcU53T0hSUmt4N0RYRnJGZHlkZUk2MGdHdkVDamJBSkg3OEpYVk54VnlCWG9QRlZCckJfZ9IBVEFVX3lxTE1yNmgwR19HQXlFckVFOGJqSk9wMEpHRDJnbGZYUWFZOUNmUHEzRGl3bXZqazFWM3hybmk3cFY1SUV1alAyMzhPUl9FNEY1NXpTT2psMw?oc=5',
  'content': "부자들은 다 안다…'1억으로 7700만원 버는' 대박 투자 - 한국경제"},
 {'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTFBWaTJrbXEweXdGYlRHa0N5bTZUNEdEM09JRVZKOFdUX1RRaW40QmtsSlJMVk44MXV4dV9jSno1bTU4VWxqSzV1UlFJeW5qWExweVAwWVFCZUpNZ9IBVkFVX3lxTE1zSXVFOVRDSm1oRFJJeEZ2Rm5tMXVTX1ZuNDlka1RpNDA0MlpCX0dCLUhUYUFpRG84VDZyVGJjaFlrcS15SVhRVWRhcnB0OWxqdlVZS1pB?oc=5',
  'content': '과기정통부, 2026년 AI 투자 방향 제시…차세대 AI 기술 개발 투자 박차 - 아주경제'},
 {'url': 'https://news.google.com/rss/articles/CBMiTkFVX3lxTE9peUV6VW14bF9oWWNFSUdjNHk5eHMzcENreWVqZEVQaG1HWmJFUEhscHNtc19yZ0lfb0JPMEVneUdaVmVtT0FxNXEyMlUyZw?oc=5',
  'content': "AI 민간투자 3위 이스라엘, '버티컬 AI'에 집중 투자 - 전자신문"},
 {'url': 'https://news.google.com/rss/articles/CBMikAFBVV

In [17]:
search_keyword.invoke({"query": "kpop"})

kpop


[{'url': 'https://news.google.com/rss/articles/CBMibEFVX3lxTE5MeFFoUGV5MVN2STZ5VWhXRjRQNnp4QklqSVFMajZFdFJTdUV1dVRMU0t5TlhGaUdjQlpwS1hxbkg4Y0xlN2VkdmdQZGVrUzYzdFpRTG5uR2xVMVVmSndpQTNXSEVmRkRBWDUxYQ?oc=5',
  'content': '삼바축구 물렀거라, ‘K-POP’이 나간다 - 재외동포신문'},
 {'url': 'https://news.google.com/rss/articles/CBMiaEFVX3lxTE9PM1RSaEVlTHZmNzAwb3d4OWNmRnNTWlRDWlJPaHVMa2tfd0txZVRUVmVaX1N0UVFlUVhxRjZpcDdhMjBkNlJ2Tkw0NWN5bGpHVVBraG1qUFBKaHRTdUlrc19DMUFjd09a0gFsQVVfeXFMTjUza0UxUDA2R0xPcHdJRnBwYmFiRXJxa2JfZ3Z1dGo0VUVhbXFJQUh2X2JpQWhmWEdmV2w4aWFHV1pmTFBYOG9HRlgzUVpnS2d5amRiS1Q5ek9oTE9PR0Ewd0pQOUZPN0hkVlpv?oc=5',
  'content': '전세계 뒤흔든 로제 ‘아파트’, ‘케이팝레이더’ 또다시 1위 - 뉴스티앤티'},
 {'url': 'https://news.google.com/rss/articles/CBMi0gNBVV95cUxOd1VVcnFIdjFVMUZOY3BicXVjdXI3OXBOdHp4c1BVZjk0WjkzNUtRcUNpTGdUY1VPSmtJU1hyUkJNOXpfRDhUZnRCN29JcUZpdlA3TG1ySV9WZ2pKUUdWWGo5SDR1TUhtckdUT1poOXIwdmhweG9fT3VicUxnNjJnN0s0a2stSEhaQzBSQVQtU1VRX3c3X1VPaXFXSnA1ZHEyRFJHQmQ1SGZHaXVtNW5yR3FwZU5uN2hzQzRkbks1YU1LS0pKNjdSOTNrYl9wRWlCYnNXRWtl